
### NCAA Tournament Simulation using historical probabilities

- https://en.wikipedia.org/wiki/NCAA_Division_I_Men%27s_Basketball_Tournament
- https://www.betfirm.com/seeds-national-championship-odds/

In [1]:
import json
import random
import pandas as pd

In [2]:
rounds = [
  '1st Round',
  '2nd Round',
  'Sweet Sixteen',
  'Regional Finals',
  'Final Four',
]

historical_probabilities = {}
with open('../../data/ncaa/historical_tourny_win_perc.json', 'rb') as tourny_percs:
    historical_probabilities = json.loads(tourny_percs.read())

In [3]:
def chunk(items, chunk_size):
    array = []
    for i in range(0, len(items), chunk_size):
        array.append(
            tuple(sorted(items[i:i + chunk_size]))
        )

    return array

def play_game(tournament_round, team1, team2):
    def get_probability(tournament_round, t1, t2):
        t1_key = str(t1)
        t2_key = str(t2)

        probability = random.random() / 10000
        if t1_key in historical_probabilities[tournament_round]:
            if t2_key in historical_probabilities[tournament_round][t1_key]:
                probability += historical_probabilities[tournament_round][t1_key][t2_key]

        return probability

    team1_probability = get_probability(tournament_round, team1, team2)
    team2_probability = get_probability(tournament_round, team2, team1)

    total_probability = team1_probability + team2_probability

    team1_probability /= total_probability
    team2_probability /= total_probability

    assert round(team1_probability + team2_probability) == 1, 'sum of probabilities does not equal 1.'

    sequence = [team1, team2]
    probabilities = [team1_probability, team2_probability]

    return random.choices(
      sequence,
      probabilities,
      k=1
    )[0]

def play_regional():
    regional = [
      [
        (1, 16),
        (8, 9),
        (5, 12),
        (4, 13),
        (6, 11),
        (3, 14),
        (7, 10),
        (2, 15),
      ]
    ]

    for tournament_round in rounds[:-1]:
        games_to_play = regional[-1]
        winners = chunk(
          [
            play_game(tournament_round, t1, t2)
            for t1, t2
            in games_to_play
          ],
          2
        )

        regional.append(winners)

    return regional


In [4]:
n = 10000
output = {
    0: { i:0 for i in range(1, 17) },
    1: { i:0 for i in range(1, 17) },
    2: { i:0 for i in range(1, 17) },
    3: { i:0 for i in range(1, 17) },
    4: { i:0 for i in range(1, 17) },
}

for _ in range(n):
    regional = play_regional()
    for i, games in enumerate(regional):
        for game in games:
            t1 = game[0]
            output[i][t1] += 1

            if len(game) > 1:
              t2 = game[1]
              output[i][t2] += 1

In [5]:
import pandas as pd

df = pd.DataFrame(output)
for i in [0, 1, 2, 3, 4]:
    df[i] = df[i] / n

df.columns = rounds
df

,1st Round,2nd Round,Sweet Sixteen,Regional Finals,Final Four
1,1.0,0.9914,0.8487,0.6851,0.3943
2,1.0,0.9346,0.6343,0.4976,0.2167
3,1.0,0.8516,0.5249,0.3197,0.1322
4,1.0,0.7850,0.4645,0.1433,0.0932
5,1.0,0.6469,0.3433,0.0606,0.0452
6,1.0,0.6269,0.2895,0.0535,0.0111
7,1.0,0.6131,0.2010,0.0522,0.0074
8,1.0,0.4948,0.0996,0.0734,0.0449
9,1.0,0.5052,0.0492,0.0256,0.0187
10,1.0,0.3869,0.1449,0.0318,0.0056


### Simulation of a Regional

In [6]:
regional_results = play_regional()

for i, regional_round in enumerate(rounds):
    results = regional_results[i]

    print(regional_round)
    print(results)
    print()


1st Round
[(1, 16), (8, 9), (5, 12), (4, 13), (6, 11), (3, 14), (7, 10), (2, 15)]

2nd Round
[(1, 8), (4, 5), (3, 6), (2, 7)]

Sweet Sixteen
[(1, 4), (2, 3)]

Regional Finals
[(2, 4)]

Final Four
[(4,)]

